# Continuous Control

---

![Reacher](doc/reacher.png)

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

Import all the dependences

In [1]:
import os
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt

from code import Config, Agent
from unityagents import UnityEnvironment

Put the environment in the config path. And select the desired env: one or twenty.

In [2]:
Config.SELECTED_ENV = 'twenty'
Config.ENABLE_TRAIN = False

In [3]:
if Config.SELECTED_ENV == 'one':
    ENV_PATH = Config.CONTINUOUS_ENV_PATH
    CHECKPOINT_PATH = Config.CHECKPOINT_CONTINUOUS_PATH
    MODEL_PATH = Config.MODEL_CONTINUOUS_PATH

elif Config.SELECTED_ENV == 'twenty':
    ENV_PATH = Config.CONTINUOUS_TWENTY_ENV_PATH
    CHECKPOINT_PATH = Config.CHECKPOINT_CONTINUOUS_TWENTY_PATH
    MODEL_PATH = Config.MODEL_CONTINUOUS_TWENTY_PATH

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
env = UnityEnvironment(file_name=ENV_PATH)

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 4. Define an agent

Is time to define an Agent to try to solve the env. In this case, the agent allow to wrap multiple agents at the same time (20) and depends of the Config.

The agent is made up of two parts:
- Actor: Uses a neural network based on politics. This unlike A2C or A3C, has a deterministic output, which is the set of most likely actions.
- Critic: Uses a neural network based on value function. 

#### 4.1 Loss
The minimization function is composed of two parts:
- **The critic (value-based):** must minimize an error function based on MSE. This is because we seek to be able to understand the environment and be able to predict the q (s, a). In other words: search the best combination of actions that maximize the reward.
![Loss_critic](doc/loss_critic.png)



- **The actor (policy-based):** must maximize originally the P(trajectory)·R(trajectory), that maximization produce to maximize the probability of the trajectory that has the best global rewards. In the DDPG case, we use the information of the critic to create an similar approach based in to maximize the q(s, a). In other words: search the best action in a specified state.
![Loss_actor](doc/loss_actor.png)

In [ ]:
agent = Agent(num_agents=num_agents, state_size=state_size, action_size=action_size, random_seed=10)

### 5. Training

In [ ]:
# ### 3. Train the Agent with DDPG
def ddpg(n_episodes=2000, t_max=1000):
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in range(1, n_episodes + 1):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations            
        scores_step = np.zeros(num_agents)             
        score = 0
        agent.reset()
        for step in range(t_max):
            actions = agent.act(states)
            env_info = env.step(actions)[brain_name]     
            next_states = env_info.vector_observations   
            rewards = env_info.rewards                   
            dones = env_info.local_done                  
            agent.step(states, actions, rewards, next_states, dones)
            states = next_states
            scores_step += rewards
            if np.any(dones):
                break

        # current score
        current_score = np.mean(scores_step)

        # current score to deque and score
        scores_deque.append(current_score)
        scores.append(current_score) 

        # Mean score of window
        mean_score = np.mean(scores_deque)

        print("===============================================")
        print("Episode: %d" % (i_episode, )) 
        print("\t- Current Score: %f (+/- %f)" % (current_score, np.std(scores_step)))
        print("\t- 100 Avg Score: %f (+/- %f)" % (mean_score, np.std(scores_deque)))

        if mean_score > 32:
            print("Average score of 30 achieved")   
            os.makedirs(CHECKPOINT_PATH, exist_ok=True)
            torch.save(agent.actor_local.state_dict(), os.path.join(CHECKPOINT_PATH, 'checkpoint_actor.pth'))
            torch.save(agent.critic_local.state_dict(), os.path.join(CHECKPOINT_PATH, 'checkpoint_critic.pth'))        
            break
    return scores

if Config.ENABLE_TRAIN:
    scores = ddpg()

    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(1, len(scores)+1), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

### 6. Results
For convenience, the process has been executed through a DDPG.py file, so the process graphs have been saved.

#### 6.1 Twenty
Below you can see how the reach twenty process has successfully overcome the barrier of +30 rewards:

```
Episode: 101
        - Current Score: 37.741499 (+/- 1.063937)
        - 100 Avg Score: 30.687429 (+/- 12.746203)
===============================================
Episode: 102
        - Current Score: 37.410999 (+/- 1.434520)
        - 100 Avg Score: 31.045569 (+/- 12.422840)
===============================================
Episode: 103
        - Current Score: 37.282499 (+/- 2.229894)
        - 100 Avg Score: 31.402674 (+/- 12.078970)
===============================================
Episode: 104
        - Current Score: 37.534999 (+/- 1.354657)
        - 100 Avg Score: 31.758544 (+/- 11.724980)
===============================================
Episode: 105
        - Current Score: 38.306999 (+/- 0.967430)
        - 100 Avg Score: 32.124954 (+/- 11.345224)
Average score of 30 achieved

```

![Results Twenty](doc/project_2_twenty_reacher.png)

### 6. Test

In [ ]:
# ### 4. Watch a Smart Agent!
agent.actor_local.load_state_dict(torch.load(os.path.join(MODEL_PATH, 'checkpoint_actor.pth')))
agent.critic_local.load_state_dict(torch.load(os.path.join(MODEL_PATH, 'checkpoint_critic.pth')))

env_info = env.reset(train_mode=False)[brain_name]
for t in range(200):
    states = env_info.vector_observations
    actions = agent.act(states, add_noise=False)
    env_info = env.step(actions)[brain_name]
    dones = env_info.local_done
    if np.any(dones):
        break 